<a href="https://colab.research.google.com/github/abhimanyuyadav627/LLM-From-Scratch/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.3 MB/s eta 0:00:00


**What is an Embedding ?**
* Embedding - **It is a mapping from discrete objects, such as words , images, or even entire documents, to points in a continous vector space.**
* While we can be using pretrained word embeddings but it is a common practice for LLMs to produce their own embeddings that are part of input layer and are updated during training.

### PREPROCESSING STEPS FOR CREATING EMBEDDINGS

##### TOKENIZING TEXT

Important Considerations:


1. When developing a simple tokenizer,
 whether we should encode whitespaces as seperate characters or just remove them depends on our application and its requirements. **Removing whitespaces reduces the memory and computing requirements. However, keeping whitespaces can be useful if we train models that are sensitive to the exact structure of the text (for example Python code, which is sensitive to indentation and spacing).**

2. Adding special tokens - to deal with unknown words that were not a part of vocabulary(not needed with BPE), to deal with situations where we need a seperator for two unrelated text sources.









In [18]:
import importlib
import tiktoken
print("tiktoken version:", importlib.metadata.version("tiktoken"))
tokenizer = tiktoken.get_encoding("gpt2")

tiktoken version: 0.6.0


In [19]:
# reading raw text from a text file.
with open("the-verdict.txt", "r") as f:
  raw_text = f.read()
encoded_text = tokenizer.encode(raw_text, allowed_special = {"<|endoftext|>"})
print("Total number of character:", len(raw_text))
print("Total number of tokens in the dataset:", len(encoded_text))

Total number of character: 20479
Total number of tokens in the dataset: 5145


In [17]:
import torch
from torch.utils.data import Dataset, Dataloader

class GPTDataset(Dataset):

  def __init__(self,text,tokenizer,max_length,stride):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt)

    for i in range(0,len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i + max_length]
      target_chunk = token_ids[i + 1:i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx], self.target_ids[idx]

tiktoken version: 0.6.0
